In [1]:
import mne
import numpy as np
from sklearn.model_selection import cross_val_score
from mne.preprocessing import ICA
import pandas as pd
from mne.time_frequency import tfr_multitaper
from mne import create_info
from mne.io import RawArray
import os
import matplotlib.pyplot as plt
# Ask for info file input
info_file_path = input("Enter the path to the info file (or leave blank if not available): ")

def read_eeg_data(file_path, info_file_path=None):
    _, file_extension = os.path.splitext(file_path)
    file_extension = file_extension.lower()
    
    if file_extension == '.edf':
        return mne.io.read_raw_edf(file_path, preload=True)
    elif file_extension == '.set':
        return mne.io.read_raw_eeglab(file_path, preload=True)
    elif file_extension in ['.txt', '.csv', '.xlsx']:
        delimiter = ','
        # Read the data without headers to infer data types
        df_without_headers = pd.read_csv(file_path, delimiter=delimiter, encoding='latin1', on_bad_lines='skip', header=None)
        
        channel_names = df_without_headers.iloc[0].tolist()
        # Infer data types for each column
        dtype_dict = df_without_headers.dtypes.to_dict()
        # Read the data again with headers and specified data types
        df = pd.read_csv(file_path, delimiter=delimiter, encoding='latin1', on_bad_lines='skip', dtype=dtype_dict)
        # Set the first row as column names
        df.columns = df.iloc[0]
        # Drop the first row
        df = df.drop(0)
        # Convert all columns to numeric except for the first row (which is now the column names)
        df = df.apply(pd.to_numeric, errors='coerce')
        # Fill NaN values with 0 and then fill with column averages
        column_averages = df.mean()
        df.fillna(0, inplace=True)
        df.fillna(column_averages, inplace=True)

        # Assuming the data is in columns and each row represents a sample
        data = df.to_numpy().T   # Transpose to have channels as rows and samples as columns
        sfreq = None
        
        if info_file_path:
            info_params = extract_info_file_params(info_file_path)
            sfreq = info_params.get('sampling_rate')
        if not sfreq:
            sfreq = float(input("Enter the sampling frequency: "))
            
        
        # Convert non-numeric values to 0
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        
        # Calculate column averages
        column_averages = df.mean()
        
        # Fill NaN values with column averages
        df.fillna(column_averages, inplace=True)
        
        for column in df.columns:
            df[column] = df[column].astype(str)
            df[column] = df[column].str.replace('[^\d.-]', '', regex=True)
            df[column] = df[column].str.replace('\.{2,}', '.', regex=True)
            df[column] = pd.to_numeric(df[column], errors='coerce').fillna(0)
           
        column_averages=df.mean()
        for column in df.columns:
            df[column].fillna(column_averages[column], inplace=True)

               
        
        info = mne.create_info(ch_names=channel_names, sfreq=sfreq, ch_types='eeg')
        raw = mne.io.RawArray(data, info)
        return raw
    else:
        raise ValueError(f"Unsupported file format: {file_extension}")

# Ask for EEG file input
eeg_file_path = input("Enter the path to the EEG file: ")

while True:
    eeg_state = input("Enter the EEG state (meditation or rest): ").lower()
    if eeg_state in ["meditation", "rest"]:
        break
    else:
        print("Invalid input. Please enter either 'meditation' or 'rest'.")

# Map EEG state to label
eeg_state_label = "Meditation" if eeg_state == "meditation" else "Rest"

while True:
    eeg_level = input("Enter the Level (experienced or novice): ").lower()
    if eeg_level in ["experienced", "novice"]:
        break
    else:
        print("Invalid input. Please enter either 'experienced' or 'novice'.")

# Map EEG state to label
eeg_level_label = "Experienced" if eeg_level == "experienced" else "Novice"



# Load EEG data
raw_data = read_eeg_data(eeg_file_path)

print(raw_data)

# Apply bandpass filtering
raw_data.filter(l_freq=4, h_freq=100, picks='eeg')

# Apply notch filtering (e.g., remove 60 Hz powerline interference)
raw_data.notch_filter(freqs=60, picks='all')






# Show all channel names in the dataset
print("Channel names in the dataset:")
print(raw_data.info['ch_names'])

# Map channel names to the appropriate nomenclature
channel_mapping = {
    'Fp1': 'Fp1', 'Fp2': 'Fp2', 'F3': 'F3', 'F4': 'F4', 'C3': 'C3',
    'C4': 'C4', 'P3': 'P3', 'P4': 'P4', 'O1': 'O1', 'O2': 'O2',
    'F7': 'F7', 'F8': 'F8', 'T7': 'T7', 'T8': 'T8', 'P7': 'P7',
    'P8': 'P8', 'Fz': 'Fz', 'Cz': 'Cz', 'Pz': 'Pz', 'FC1': 'FC1',
    'FC2': 'FC2', 'CP1': 'CP1', 'CP2': 'CP2', 'FC5': 'FC5', 'FC6': 'FC6',
    'CP5': 'CP5', 'CP6': 'CP6', 'FT9': 'FT9', 'FT10': 'FT10',
    'TP9': 'TP9', 'TP10': 'TP10', 'POz': 'POz', 'ECG': 'ECG',
    'EXG1': 'EXG1', 'EXG2': 'EXG2', 'EXG3': 'EXG3', 'EXG4': 'EXG4',
    'EXG5': 'EXG5', 'EXG6': 'EXG6', 'EXG7': 'EXG7', 'EXG8': 'EXG8',
    'Status': 'Status'
}



# Ask for info file input
info_file_path = input("Enter the path to the info file (or leave blank if not available): ")

def extract_info_file_params(info_file_paAth):
    """
    Extract parameters from the info file.

    Args:
    info_file_path (str): Path to the info file.

    Returns:
    dict: Dictionary containing extracted parameters.
    """
    params = {}
    if info_file_path:
        with open(info_file_path, 'r') as file:
            lines = file.readlines()

        for line in lines:
            line = line.strip()  # Remove leading and trailing whitespace
            if line.startswith('Total number of channels:'):
                params['total_channels'] = int(line.split(':')[1].strip())
            elif line.startswith('EEG sampling rate:'):
                params['sampling_rate'] = float(line.split(':')[1].split()[0])
            elif line.startswith('Number of EEG channels:'):
                params['num_eeg_channels'] = int(line.split(':')[1].strip())
            elif line.startswith('Accelerometer data:'):
                params['accelerometer_data'] = line.split(':')[1].strip() == 'ON'
            elif line.startswith('Accelerometer sampling rate:'):
                params['accelerometer_sampling_rate'] = float(line.split(':')[1].strip().split()[0])
            elif line.startswith('Number of channels of Accelerometer:'):
                params['num_accelerometer_channels'] = int(line.split(':')[1].strip())
            elif line.startswith('Reference channels:'):
                params['reference_channels'] = int(line.split(':')[1].strip())

    return params


sampling_rate = None
accelerometer_data = False
accelerometer_sampling_rate = None
accelerometer_units = None
if info_file_path:
    info_params = extract_info_file_params(info_file_path)
    sampling_rate = info_params.get('sampling_rate')
    accelerometer_data = info_params.get('accelerometer_data', False)
    accelerometer_sampling_rate = info_params.get('accelerometer_sampling_rate')
    accelerometer_units = info_params.get('accelerometer_units')
    num_accelerometer_channels = info_params.get('num_accelerometer_channels')
    

# Set montage according to info file or user preference
if info_file_path:
    info_params = extract_info_file_params(info_file_path)
    montage = info_params.get('montage_channels')
    if not montage:
        montage_choice = input("Montage not found in info file. Do you want to use 10-10 montage? (y/n): ")
        if montage_choice.lower() == 'y':
            montage = 'standard_1005'
        else:
            montage = 'standard_1020'
else:
    montage = 'standard_1020'

if montage:
    try:
        montage = mne.channels.make_standard_montage(montage, head_size=1.0)
        raw_data.set_montage(montage)
    except ValueError as e:
        print("Error setting montage:", e)
        print("Setting montage without positions...")
        raw_data.set_montage(montage, on_missing='ignore')
else:
    print("Montage not specified. Setting default 10-20 montage.")
    montage = 'standard_1020'
    raw_data.set_montage(montage, on_missing='ignore')


    
info_params = extract_info_file_params(info_file_path)
print("Extracted info parameters:", info_params)  # Move this line here    
    
    #meditation    #rest    #novice 

Enter the path to the info file (or leave blank if not available): 
Enter the path to the EEG file: c_rest_n6.txt
Enter the EEG state (meditation or rest): rest
Enter the Level (experienced or novice): novice


C:\Users\user\AppData\Local\Temp\ipykernel_13620\1793205669.py:25: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_without_headers = pd.read_csv(file_path, delimiter=delimiter, encoding='latin1', on_bad_lines='skip', header=None)


Enter the sampling frequency: 512
Creating RawArray with float64 data, n_channels=24, n_times=620546
    Range : 0 ... 620545 =      0.000 ...  1212.002 secs
Ready.
<RawArray | 24 x 620546 (1212.0 s), ~113.6 MB, data loaded>
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 845 samples (1.650 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.1s finished


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 3381 samples (6.604 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.1s finished


Channel names in the dataset:
['FP1', 'FP2', 'F7', 'F3', 'Fz', 'F4', 'F8', 'T3', 'C3', 'Cz', 'C4', 'T4', 'T5', 'P3', 'PZ', 'P4', 'T6', 'O1', 'O2', 'A1', 'A2', 'SensorCEOG', 'SensorDEOG', 'Events']
Enter the path to the info file (or leave blank if not available): 
Error setting montage: DigMontage is only a subset of info. There are 6 channel positions not present in the DigMontage. The channels missing from the montage are:

['FP1', 'FP2', 'PZ', 'SensorCEOG', 'SensorDEOG', 'Events'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.
Setting montage without positions...
Extracted info parameters: {}


In [2]:
#20240503172253_ShashankT_Meditation.info
# Show all channel names in the dataset according to nomenclature
print("Channel names in the dataset:")
for idx, channel in enumerate(raw_data.info['ch_names']):
    print(f"  {idx + 1}. {channel_mapping.get(channel, channel)}")


info_params = extract_info_file_params(info_file_path)
num_accelerometer_channels = info_params.get('num_accelerometer_channels')
# Ask for the names of accelerometer channels available in the raw EDF file
accelerometer_channel_names = []
if num_accelerometer_channels:
    print(f"Found {num_accelerometer_channels} accelerometer channels in the info file.")
    for i in range(num_accelerometer_channels):
        channel_name = input(f"Enter the name of accelerometer channel {i+1}: ")
        accelerometer_channel_names.append(channel_name)

# Ask if any channels need to be removed
remove_channels = input("Do you want to remove any channels? (y/n): ")
if remove_channels.lower() == 'y':
    # Ask for channels to remove
    channels_to_remove = input("Enter the channels to remove (separated by commas): ").split(',')
else:
    channels_to_remove = []

eeg_channels = [ch for ch in raw_data.ch_names if ch not in channels_to_remove and ch not in accelerometer_channel_names]


# Ask if the user wants to edit channel names
edit_channel_names = input("Do you want to edit any channel names? (y/n): ")
if edit_channel_names.lower() == 'y':
    while True:
        channel_to_edit = input("Enter the channel name you want to edit (leave blank to continue): ")
        if not channel_to_edit:
            break
        new_name = input(f"Enter the new name for {channel_to_edit}: ")
        raw_data.info['ch_names'][raw_data.info['ch_names'].index(channel_to_edit)] = new_name

# Detect reference channels
reference_channels = []
if info_file_path:
    info_params = extract_info_file_params(info_file_path)
    reference_channels_info = info_params.get('reference_channels')
    if reference_channels_info:
        reference_channels = reference_channels_info.split(',')

if not reference_channels:
    print("Reference channels not found in the info file.")
    print("Available channel names:")
    raw_data = read_eeg_data(eeg_file_path)
    for idx, channel in enumerate(raw_data.info['ch_names']):
        print(f"  {idx + 1}: {channel}")
    while True:
        reference_channel_names = input("Enter the names of the reference channels (comma-separated): ")
        if reference_channel_names:
            reference_channel_names = [name.strip() for name in reference_channel_names.split(',')]
            if all(name in raw_data.info['ch_names'] for name in reference_channel_names):
                reference_channels = reference_channel_names
                break
            else:
                print("Invalid input. Please enter valid channel names (e.g., Fp1, Fp2, Cz).")
        else:
            print("No reference channels provided. Skipping re-referencing.")
            reference_channels = []
            break

print("Selected reference channels:", reference_channels)

# Check if reference channels are present in the data
if reference_channels:
    print("Reference channels detected. Applying average re-referencing.")
    raw_data.set_eeg_reference(ref_channels='average', projection=False)
    # Remove reference channels from the raw data
    raw_data.drop_channels(reference_channels)
else:
    print("No reference channels detected. Keeping original referencing.")



eeg_channels = [ch for ch in raw_data.ch_names if ch not in channels_to_remove and ch not in accelerometer_channel_names and ch not in reference_channel_names]
all_channels = eeg_channels + accelerometer_channel_names + reference_channels
not_eeg_channels = [ch for ch in raw_data.ch_names if ch not in eeg_channels]


#T3,T4,T5,T6,P3,P4,O1,O2,C3,C4,Events,PZ,Fz,Cz,FP1,FP2T3,T4,T5,T6,P3,P4,O1,O2,C3,C4,Events,PZ,Fz,Cz,FP1,FP2

#T3,T4,T5,T6,P3,P4,O1,O2,C3,C4,Events,PZ,Fz,Cz,FP1,FP2
#F3,F4,F7,F8,T5,T6,PZ,Fz,Cz,Events,C3,C4,O1,O2

Channel names in the dataset:
  1. FP1
  2. FP2
  3. F7
  4. F3
  5. Fz
  6. F4
  7. F8
  8. T3
  9. C3
  10. Cz
  11. C4
  12. T4
  13. T5
  14. P3
  15. PZ
  16. P4
  17. T6
  18. O1
  19. O2
  20. A1
  21. A2
  22. SensorCEOG
  23. SensorDEOG
  24. Events
Do you want to remove any channels? (y/n): y
Enter the channels to remove (separated by commas): F3,F4,F7,F8,T5,T6,PZ,Fz,Cz,Events,C3,C4,O1,O2
Do you want to edit any channel names? (y/n): n
Reference channels not found in the info file.
Available channel names:


C:\Users\user\AppData\Local\Temp\ipykernel_13620\1793205669.py:25: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_without_headers = pd.read_csv(file_path, delimiter=delimiter, encoding='latin1', on_bad_lines='skip', header=None)


Enter the sampling frequency: 512
Creating RawArray with float64 data, n_channels=24, n_times=620546
    Range : 0 ... 620545 =      0.000 ...  1212.002 secs
Ready.
  1: FP1
  2: FP2
  3: F7
  4: F3
  5: Fz
  6: F4
  7: F8
  8: T3
  9: C3
  10: Cz
  11: C4
  12: T4
  13: T5
  14: P3
  15: PZ
  16: P4
  17: T6
  18: O1
  19: O2
  20: A1
  21: A2
  22: SensorCEOG
  23: SensorDEOG
  24: Events
Enter the names of the reference channels (comma-separated): A1,A2
Selected reference channels: ['A1', 'A2']
Reference channels detected. Applying average re-referencing.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


In [3]:
from mne.preprocessing import ICA
import numpy as np

snr_values = []
best_n_components = None
best_ica = None
best_snr = -np.inf

# Assuming raw_data, accelerometer_channel_names, eeg_channels, and channels_to_remove are defined

# Ask user if they want to use EOG-based artifact rejection
use_eog_rejection = input("Do you want to use EOG-based artifact rejection? (yes/no): ").lower()

# Initialize eeg_raw_data
eeg_raw_data = raw_data.copy().pick_channels(eeg_channels)

if use_eog_rejection == 'yes':
    # Your code for EOG-based artifact rejection
    if 'EOG' in raw_data.info['ch_names']:
        eog_indices = [raw_data.ch_names.index(ch) for ch in raw_data.info['ch_names'] if 'EOG' in ch]
    else:
        eog_channel_names = input("Enter EOG channel names separated by commas: ").split(',')
        raw_data_with_eog = raw_data.copy().pick_channels(eog_channel_names + eeg_channels)
        eog_indices = [raw_data_with_eog.ch_names.index(ch.strip()) for ch in eog_channel_names]

    if eog_indices:
        # Exclude EOG channels from EEG channels for ICA
        eeg_channels_for_ica = [ch for ch in raw_data_with_eog.ch_names if ch not in eog_channel_names]
        raw_data_with_eeg = raw_data_with_eog.copy().pick_channels(eeg_channels_for_ica)
        
        # Apply ICA only to EEG channels
        print("ICA with EOG data is applied.")
        for n_components in range(2, len(raw_data_with_eeg.info['ch_names']) + 1):
            ica = ICA(n_components=n_components)
            ica.fit(raw_data_with_eeg)

            reconstructed_data = ica.apply(raw_data_with_eeg)
            residual = raw_data_with_eeg.get_data() - reconstructed_data.get_data()
            noise_powers = np.mean(np.square(residual))
            signal_powers = np.mean(np.square(reconstructed_data.get_data()))
            snr = 10 * np.log10(signal_powers / noise_powers)
            mean_snr = np.mean(snr)

            if mean_snr > best_snr:
                best_snr = mean_snr
                best_n_components = n_components
                best_ica = ica

        # Discard components highly correlated with EOG channels
        eog_threshold = 0.6
        eog_inds, scores = ica.find_bads_eog(raw_data_with_eog, ch_name=eog_channel_names)
        ica.exclude.extend(eog_inds)
        for idx, score in enumerate(scores):
            if np.any(np.abs(score) > eog_threshold):
                ica.exclude.append(idx)

        print("Artifact detection using EOG channels completed.")

    else:
        print("No valid EOG channels found. EOG-based artifact rejection cannot be applied.")

elif accelerometer_channel_names:
    # Check for accelerometer sampling rate
    if 'accelerometer_sampling_rate' in raw_data.info:
        accel_sfreq = raw_data.info['accelerometer_sampling_rate']
    else:
        accel_sfreq = float(input("Enter the sampling rate of the accelerometer data: "))
    
    desired_ch = eeg_channels +accelerometer_channel_names
    # Extract accelerometer data
    raw_data_with_accelerometer = raw_data.copy().pick_channels(desired_ch, ordered=True)
    accel_data = raw_data.copy().pick_channels(accelerometer_channel_names).get_data().T

    # Function to identify movement periods
    def identify_movement_periods(accel_data, sfreq, threshold=0.5):
        # Calculate the magnitude of the acceleration vector
        accel_magnitude = np.sqrt(np.sum(accel_data ** 2, axis=1))
        # Identify periods where the magnitude exceeds the threshold
        movement_periods = accel_magnitude > threshold
        return movement_periods

    # Identify movement periods
    movement_periods = identify_movement_periods(accel_data, accel_sfreq)

    # Create annotations for the movement periods
    movement_onsets = np.where(movement_periods)[0] / accel_sfreq
    movement_durations = np.ones_like(movement_onsets) / accel_sfreq
    movement_descriptions = ['BAD_mov' for _ in range(len(movement_onsets))]

    annotations = mne.Annotations(movement_onsets, movement_durations, movement_descriptions)
    raw_data_with_accelerometer.set_annotations(annotations)

    # Optionally, reject bad segments or interpolate bad data
    # Rejecting bad segments:
    raw_data_cleaned = eeg_raw_data.copy().interpolate_bads(reset_bads=True)

    # Apply ICA to EEG channels with accelerometer data
    
    print("ICA with Accelerometer data is applied.")
    n_components_range = range(2, len(eeg_raw_data.info['ch_names']) + 1)
    for n_components in range(2, len(eeg_raw_data.info['ch_names']) + 1):
        ica = ICA(n_components=n_components)
        ica.fit(eeg_raw_data)
    
        reconstructed_data = ica.apply(eeg_raw_data)
        residual = eeg_raw_data.get_data() - reconstructed_data.get_data()
        noise_powers = np.mean(np.square(residual))
        signal_powers = np.mean(np.square(reconstructed_data.get_data()))
        snr = 10 * np.log10(signal_powers / noise_powers)
        mean_snr = np.mean(snr)

        if mean_snr > best_snr:
            best_snr = mean_snr
            best_n_components = n_components
            best_ica = ica

        raw_data_cleaned = best_ica.apply(eeg_raw_data)
    print("Best n_components:", best_n_components)

else:
    # Your code for normal ICA without EOG or accelerometer
    for n_components in range(2, len(eeg_raw_data.info['ch_names'])+1):
        ica = ICA(n_components=n_components)
        ica.fit(eeg_raw_data)

        reconstructed_data = ica.apply(eeg_raw_data)
        residual = eeg_raw_data.get_data() - reconstructed_data.get_data()
        noise_powers = np.mean(np.square(residual))
        signal_powers = np.mean(np.square(reconstructed_data.get_data()))
        snr = 10 * np.log10(signal_powers / noise_powers)
        mean_snr = np.mean(snr)

        if mean_snr > best_snr:
            best_snr = mean_snr
            best_n_components = n_components
            best_ica = ica
        
        raw_data_cleaned = best_ica.apply(eeg_raw_data)
    print("Best n_components:", best_n_components)
    
if best_ica is not None:
    if accelerometer_channel_names:
        # ICA with accelerometer data
        raw_data_cleaned = best_ica.apply(eeg_raw_data)
    elif use_eog_rejection == 'yes':
        # ICA with EOG data
        raw_data_cleaned = best_ica.apply(raw_data_with_eeg)
    else:
        raw_data_cleaned = best_ica.apply(eeg_raw_data)
else:
    print("Error: best_ica is None.")
#SensorCEOG,SensorDEOG

Do you want to use EOG-based artifact rejection? (yes/no): yes
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Enter EOG channel names separated by commas: SensorCEOG,SensorDEOG
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
ICA with EOG data is applied.
Fitting ICA to data using 6 channels (please be patient, this may take a while)
Selecting by number: 2 components


C:\Users\user\AppData\Local\Temp\ipykernel_13620\1306790683.py:23: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw_data_with_eog = raw_data.copy().pick_channels(eog_channel_names + eeg_channels)
C:\Users\user\AppData\Local\Temp\ipykernel_13620\1306790683.py:35: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(raw_data_with_eeg)


Fitting ICA took 1.7s.
Applying ICA to Raw instance
    Transforming to ICA space (2 components)
    Zeroing out 0 ICA components
    Projecting back using 6 PCA components
Fitting ICA to data using 6 channels (please be patient, this may take a while)
Selecting by number: 3 components


C:\Users\user\AppData\Local\Temp\ipykernel_13620\1306790683.py:41: RuntimeWarning: divide by zero encountered in scalar divide
  snr = 10 * np.log10(signal_powers / noise_powers)
C:\Users\user\AppData\Local\Temp\ipykernel_13620\1306790683.py:35: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(raw_data_with_eeg)


Fitting ICA took 27.1s.
Applying ICA to Raw instance
    Transforming to ICA space (3 components)
    Zeroing out 0 ICA components
    Projecting back using 6 PCA components
Fitting ICA to data using 6 channels (please be patient, this may take a while)
Selecting by number: 4 components


...

Fitting ICA took 44.6s.
Applying ICA to Raw instance
    Transforming to ICA space (6 components)
    Zeroing out 0 ICA components
    Projecting back using 6 PCA components
Using EOG channels: SensorCEOG, SensorDEOG
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5120 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passban

C:\Users\user\AppData\Local\Temp\ipykernel_13620\1306790683.py:41: RuntimeWarning: divide by zero encountered in scalar divide
  snr = 10 * np.log10(signal_powers / noise_powers)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5120 samples (10.000 s)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 5120 sam

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


    Projecting back using 6 PCA components


In [4]:

# Ask for the epoch duration input
epoch_duration = float(input("Enter the duration of each epoch in seconds (default is 1 second): ") or 1.0)

sfreq = raw_data_cleaned.info['sfreq']
# Calculate the number of samples per epoch
samples_per_epoch = int(epoch_duration * sfreq)

# Calculate the number of epochs
num_epochs = int(len(raw_data_cleaned) / samples_per_epoch)

# Create events at the beginning of each epoch
events = np.array([[i * samples_per_epoch, 0, 1] for i in range(num_epochs)])

# Define event ID (assuming event ID is 1)
event_id = 1

# Define the time window for each epoch
tmin = 0
tmax = epoch_duration

# Select EEG channels for epoching
picks_eeg = mne.pick_types(raw_data_cleaned.info, meg=False, eeg=True, eog=False)

# Print the names of the channels in picks_eeg
eeg_channel_names = [raw_data_cleaned.ch_names[pick] for pick in picks_eeg]
print("EEG channels selected for epoching:", eeg_channel_names)

# Ask for the rejection threshold input
reject_threshold_input = input("Enter the rejection threshold for epochs (leave blank for default: 100uV): ")
if reject_threshold_input:
    reject_threshold = float(reject_threshold_input)
else:
    reject_threshold = 100  # Default threshold

# Create epochs
epochs = mne.Epochs(raw_data_cleaned, events, event_id, tmin, tmax, baseline=None,
                    reject=dict(eeg=reject_threshold), picks=picks_eeg, preload=True,
                    detrend=1, reject_by_annotation=False)

# Check the number of remaining epochs
num_epochs = len(epochs)
if num_epochs == 0:
    raise ValueError("All epochs were rejected. Please adjust the rejection threshold or check your data.")

print(f"Number of epochs after rejection: {num_epochs}")

# Define frequency range and multitaper parameters
freqs = np.arange(4, 100, 0.5)
n_cycles = freqs / 6


# Calculate time-frequency power
tfr_power = tfr_multitaper(epochs, freqs=freqs, n_cycles=n_cycles,
                           time_bandwidth=2.0, return_itc=False,
                           picks=picks_eeg, average=False)

# Extract power values
power_values = tfr_power.data.astype(np.float32)

# Define frequency ranges for PSD and band power calculation
alpha_range = (8, 12)
beta_range = (13, 30)
gamma_range = (31, 100)  # Adjusted gamma range
theta_range = (4, 7)

# Calculate indices for frequency ranges
alpha_freq_indices = np.where((freqs >= alpha_range[0]) & (freqs <= alpha_range[1]))
theta_freq_indices = np.where((freqs >= theta_range[0]) & (freqs <= theta_range[1]))
beta_freq_indices = np.where((freqs >= beta_range[0]) & (freqs <= beta_range[1]))
gamma_freq_indices = np.where((freqs >= gamma_range[0]) & (freqs <= gamma_range[1]))

# Initialize arrays to store calculated metrics
num_channels = len(raw_data_cleaned.ch_names)
slope_values = np.zeros(num_channels)
itf_values = np.zeros(num_channels)
iaf_values = np.zeros(num_channels)
igf_values = np.zeros(num_channels)
ibf_values = np.zeros(num_channels)
iap_values = np.zeros(num_channels)
igp_values = np.zeros(num_channels)
ibp_values = np.zeros(num_channels)
itp_values = np.zeros(num_channels)
mean_psd_alpha_channels = np.zeros(num_channels)
mean_psd_beta_channels = np.zeros(num_channels)
mean_psd_gamma_channels = np.zeros(num_channels)
mean_psd_theta_channels = np.zeros(num_channels)
theta_alpha_ratio = np.zeros(num_channels)
theta_gamma_ratio = np.zeros(num_channels)
variance_per_channel = np.zeros(num_channels)
entropy_per_channel =   np.zeros(num_channels)


epsilon = 1e-10



# Iterate over each channel
for channel_idx in range (num_channels):
    power_channel = power_values[:, channel_idx, :, :]
        
    
    # Calculate PSD and band power for each frequency band
    mean_psd_alpha_channels[channel_idx] = np.mean(np.mean(power_channel[:, alpha_freq_indices, :], axis=(0, 2)))
    mean_psd_beta_channels[channel_idx] = np.mean(np.mean(power_channel[:, beta_freq_indices, :], axis=(0, 2)))
    mean_psd_gamma_channels[channel_idx] = np.mean(np.mean(power_channel[:, gamma_freq_indices, :], axis=(0, 2)))
    mean_psd_theta_channels[channel_idx] = np.mean(np.mean(power_channel[:, theta_freq_indices, :], axis=(0, 2)))

    # Calculate indices of maximum power within frequency bands
    max_power_theta_indices = np.argmax(power_channel[:, theta_freq_indices, :], axis=2)
    max_power_alpha_indices = np.argmax(power_channel[:, alpha_freq_indices, :], axis=2)
    max_power_beta_indices = np.argmax(power_channel[:, beta_freq_indices, :], axis=2)
    max_power_gamma_indices = np.argmax(power_channel[:, gamma_freq_indices, :], axis=2)

    # Calculate ITF, IAF, IBF, IGF values
    itf_values[channel_idx] = np.mean(freqs[theta_freq_indices][max_power_theta_indices])
    iaf_values[channel_idx] = np.mean(freqs[alpha_freq_indices][max_power_alpha_indices])
    ibf_values[channel_idx] = np.mean(freqs[beta_freq_indices][max_power_beta_indices])
    igf_values[channel_idx] = np.mean(freqs[gamma_freq_indices][max_power_gamma_indices])

    # Calculate maximum power values within frequency bands
    iap_values[channel_idx] = np.max(power_channel[:, alpha_freq_indices, :])
    ibp_values[channel_idx] = np.max(power_channel[:, beta_freq_indices, :])
    itp_values[channel_idx] = np.max(power_channel[:, theta_freq_indices, :])
    igp_values[channel_idx] = np.max(power_channel[:, gamma_freq_indices, :])

    # Calculate slope
    log_freqs = np.log(freqs)
    log_power = np.log(power_channel + epsilon) #small offset to remove log0 problem

    log_freqs_alpha = log_freqs[alpha_freq_indices]
    log_power_alpha = log_power[:, :, alpha_freq_indices]

    log_power_alpha_reshaped = log_power_alpha.reshape(-1, log_power_alpha.shape[-1])

    X = np.column_stack((np.ones(log_freqs_alpha.shape), log_freqs_alpha))
    results = np.linalg.lstsq(X, log_power_alpha_reshaped.T, rcond=None)
    slope = results[0][1]

    slope_values[channel_idx] = slope[0]





for channel_idx in range(num_channels):
    #Calculate Theta/Alpha ratio for each channel
    theta_alpha_ratio[channel_idx] = mean_psd_theta_channels[channel_idx] / mean_psd_alpha_channels[channel_idx]
        #Calculate Theta/Alpha ratio for each channel
    theta_gamma_ratio[channel_idx] = mean_psd_theta_channels[channel_idx] / mean_psd_gamma_channels[channel_idx]

    
for channel_idx in range(num_channels):
    power_channel = power_values[:, channel_idx, :, :]

    # Calculate variance and entropy for each channel separately across all epochs
    variance_per_channel[channel_idx] = np.mean(np.var(power_channel, axis=(0, 2)))
    entropy_per_channel[channel_idx] = -np.mean(np.sum(power_channel * np.log(power_channel+epsilon), axis=(0, 2)))

    



# Prepare results for DataFrame
results_dict = {
    'Slope': slope_values,
    'ITF': itf_values,
    'IAF': iaf_values,
    'IGF': igf_values,
    'IBF': ibf_values,
    'IAP': iap_values,
    'ITP': itp_values,
    'IGP': igp_values,
    'IBP': ibp_values,
    'Mean_PSD_Alpha': mean_psd_alpha_channels,
    'Mean_PSD_Beta': mean_psd_beta_channels,
    'Mean_PSD_Gamma': mean_psd_gamma_channels,
    'Mean_PSD_Theta': mean_psd_theta_channels,
    'Theta_Alpha_Ratio': theta_alpha_ratio,
    'Theta_Gamma_Ratio': theta_gamma_ratio,
    'Variance': variance_per_channel,
    'Entropy': entropy_per_channel
}


# Define the path for the CSV file
results_csv_file = 'Frameworkfeatures2.csv'

# Extract the filename from the eeg_file_path
filename = os.path.basename(eeg_file_path)

# Create a dictionary to store data for each column
data_dict = {'Filename': filename, 'EEG_State': eeg_state_label, 'Level': eeg_level_label} 

# Append EEG channel features to the DataFrame
for channel_idx in range(num_channels):
    for feature_name, feature_value in results_dict.items():
        # Construct column name as feature name + EEG channel name
        column_name = f'{feature_name}_channel{channel_idx+1}'
        data_dict[column_name] = feature_value[channel_idx]

# Append the data to the DataFrame
results_df = pd.DataFrame(data_dict, index=[0])

# Check if the CSV file exists
if not os.path.exists(results_csv_file):
    # If the file does not exist, create a new file and write the DataFrame
    results_df.to_csv(results_csv_file, index=False)
    print("Results saved to:", results_csv_file)
else:
    # If the file already exists, append the new data to the existing file
    results_df.to_csv(results_csv_file, mode='a', header=False, index=False)
    print("Results appended to:", results_csv_file)


Enter the duration of each epoch in seconds (default is 1 second): 
EEG channels selected for epoching: ['FP1', 'FP2', 'T3', 'T4', 'P3', 'P4']
Enter the rejection threshold for epochs (leave blank for default: 100uV): 
Not setting metadata
1212 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1212 events and 513 original time points ...
    Rejecting  epoch based on EEG : ['FP1', 'FP2', 'T3', 'T4', 'P3', 'P4']
    Rejecting  epoch based on EEG : ['T3']
    Rejecting  epoch based on EEG : ['T3']
    Rejecting  epoch based on EEG : ['T3']
    Rejecting  epoch based on EEG : ['T3']
    Rejecting  epoch based on EEG : ['P3']
    Rejecting  epoch based on EEG : ['FP1', 'FP2', 'T3', 'T4', 'P3', 'P4']
7 bad epochs dropped
Number of epochs after rejection: 1205


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    9.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   12.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   18.7s finished


Not setting metadata
Results appended to: Frameworkfeatures2.csv


In [5]:
import pandas as pd
import matlpotlib.pyplot as plt

ModuleNotFoundError: No module named 'matlpotlib'

In [ ]:
# Read the CSV file
df = pd.read_csv("Frameworkfeatures2.csv")

# Show EEG channel names
print("EEG channel names:")
for idx, channel in enumerate(df.columns):
    print(f"  {idx + 1}. {channel}")

# Function to get channel pairs for a specific region
def get_channel_pairs(region_name):
    pairs_input = input(f"Enter EEG channel pairs for {region_name} region (channel names separated by comma) (e.g., 'Fz,Cz P3,P4'): ")
    return [tuple(pair.split(',')) for pair in pairs_input.split()]

# Dictionary to store region names and their respective channel pairs
region_pairs = {
    "Frontal": [],
    "Parietal": [],
    "Central": [],
    "Temporal": [],
    "Occipital": []
}

# Get channel pairs for each region
for region_name in region_pairs.keys():
    region_pairs[region_name] = get_channel_pairs(region_name)

# List of features to process
features_to_process = [
    "Slope", "ITF", "IAF", "IGF", "IBF", "IAP", "ITP", "IGP", "IBP",
    "Mean_PSD_Alpha", "Mean_PSD_Beta", "Mean_PSD_Gamma", "Mean_PSD_Theta",
    "Theta_Alpha_Ratio", "Theta_Gamma_Ratio", "Variance", "Entropy"
]

# Iterate over features and channel pairs to compute mean values
for feature in features_to_process:
    for region_name, pairs in region_pairs.items():
        if pairs:  # Check if channel pairs are provided for the region
            for pair in pairs:
                # Select channels for the pair
                channels = [f"{feature}_{channel}" for channel in pair if f"{feature}_{channel}" in df.columns]

                # If channels exist for the pair
                if channels:
                    # Filter out non-numeric values
                    numeric_mask = df[channels].apply(pd.to_numeric, errors='coerce').notnull().all(axis=1)

                    # Compute mean for all channels in the pair
                    df[f"Mean_{feature}_{region_name}"] = df[channels].mean(axis=1)

                    # Drop the original columns
                    df.drop(channels, axis=1, inplace=True)

# Save the modified DataFrame to a new CSV file
df.to_csv("ModifiedFrameworkfeatures2.csv", index=False)




# Read the modified CSV file
df = pd.read_csv("ModifiedFrameworkfeatures2.csv")

# Define conditions with names
conditions = {
    "Experienced Rest": ("Experienced", "Rest", "experienced-rest"),
    "Experienced Meditation": ("Experienced", "Meditation", "experienced-meditation"),
    "Novice Rest": ("Novice", "Rest", "novice-rest"),
    "Novice Meditation": ("Novice", "Meditation", "novice-meditation")
}

# Initialize an empty DataFrame to store mean values
condition_means_df = pd.DataFrame(columns=list(df.columns) + ['Condition'])

# Calculate mean for each condition
for condition, (level, eeg_state, condition_name) in conditions.items():
    # Filter rows based on condition
    condition_df = df[(df['Level'] == level) & (df['EEG_State'] == eeg_state)]
    # Calculate mean
    condition_mean = condition_df.mean(axis=0)
    # Set condition name
    condition_mean['Condition'] = condition_name
    # Append mean values to condition_means_df
    condition_means_df = condition_means_df.append(condition_mean, ignore_index=True)

# Drop the columns "Filename", "EEG_state", and "Level"
condition_means_df.drop(["Filename", "EEG_State", "Level"], axis=1, inplace=True)

# Reorder columns to have "Condition" as the first column
cols = condition_means_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
condition_means_df = condition_means_df[cols]

# Save the DataFrame with condition means to a new CSV file
condition_means_df.to_csv("ConditionMeansFrameworkfeatures2.csv", index=False)


# Read the CSV file with condition means
df = pd.read_csv("ConditionMeansFrameworkfeatures2.csv")

# Extract condition names
conditions = df['Condition']

# Drop the 'Condition' column
df = df.drop(columns=['Condition'])


# Set up plot grid
num_features = len(df.columns)
num_cols = 2  # Number of columns in the plot grid
num_rows = (num_features + 1) // num_cols  # Calculate number of rows needed
plt.figure(figsize=(15, 5 * num_rows))

# Iterate over columns (features)
for i, column in enumerate(df.columns, 1):
    # Create a subplot for the current feature
    plt.subplot(num_rows, num_cols, i)
    
    # Plot bar graph for the current feature
    plt.bar(conditions, df[column], color=['blue', 'orange', 'green', 'red'])
    
    # Set plot title
    plt.title(column)
    
    # Set plot properties
    plt.xlabel('Conditions')
    plt.ylabel('Value')
    plt.xticks(rotation=45)
    plt.tight_layout()

# Show plots
plt.show()


In [ ]:
import pandas as pd

# Read the modified CSV file
df = pd.read_csv("ModifiedFrameworkfeatures.csv")

# Define conditions with names
conditions = {
    "Experienced Rest": ("Experienced", "Rest", "experienced-rest"),
    "Experienced Meditation": ("Experienced", "Meditation", "experienced-meditation"),
    "Novice Rest": ("Novice", "Rest", "novice-rest"),
    "Novice Meditation": ("Novice", "Meditation", "novice-meditation")
}

# Initialize an empty DataFrame to store max values
condition_max_df = pd.DataFrame(columns=list(df.columns) + ['Condition'])

# Calculate max for each condition
for condition, (level, eeg_state, condition_name) in conditions.items():
    # Filter rows based on condition
    condition_df = df[(df['Level'] == level) & (df['EEG_State'] == eeg_state)]
    # Calculate max
    condition_max = condition_df.max(axis=0)
    # Set condition name
    condition_max['Condition'] = condition_name
    # Append max values to condition_max_df
    condition_max_df = condition_max_df.append(condition_max, ignore_index=True)

# Drop the columns "Filename", "EEG_state", and "Level"
condition_max_df.drop(["Filename", "EEG_State", "Level"], axis=1, inplace=True)

# Reorder columns to have "Condition" as the first column
cols = condition_max_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
condition_max_df = condition_max_df[cols]

# Save the DataFrame with condition max values to a new CSV file
condition_max_df.to_csv("ConditionMaxFrameworkfeatures.csv", index=False)


In [ ]:
import numpy as np
import pandas as pd

# Load data
condition_means_df = pd.read_csv("ConditionMaxFrameworkfeatures.csv", index_col=0)

# Define the conditions
conditions = {
    "Experienced Rest": ("Experienced", "Rest", "experienced-rest"),
    "Experienced Meditation": ("Experienced", "Meditation", "experienced-meditation"),
    "Novice Rest": ("Novice", "Rest", "novice-rest"),
    "Novice Meditation": ("Novice", "Meditation", "novice-meditation")
}

# Ask for input regarding the threshold percentage
threshold_percentage = float(input("Enter the threshold percentage (between 0 and 100): "))

# Convert the threshold percentage to a scale factor
threshold_scale_factor = threshold_percentage / 100

# Min-Max scaling
condition_means_df_scaled = (condition_means_df - condition_means_df.min()) / (condition_means_df.max() - condition_means_df.min())

# Perform scaling without normalization
condition_means_df_scaled *= 100  # Multiplying by 100 to keep values within a reasonable range

# Calculate the range of values for each feature
feature_ranges = condition_means_df_scaled.max() - condition_means_df_scaled.min()

# Define specified conditions to compare
specified_conditions = [
    ("Experienced Rest", "Experienced Meditation"),
    ("Experienced Rest", "Novice Rest"),
    ("Novice Rest", "Novice Meditation"),
    ("Novice Meditation", "Experienced Meditation")
]

# Find differences exceeding threshold or showing significant difference
significant_differences = []  # Store unique combinations of differences
for feature in condition_means_df.columns:
    for cond1, cond2 in specified_conditions:
        diff = condition_means_df_scaled.loc[conditions[cond1][2], feature] - condition_means_df_scaled.loc[conditions[cond2][2], feature]
        threshold = feature_ranges[feature] * threshold_scale_factor
        if abs(diff) >= threshold:
            significant_differences.append((feature, cond1, cond2, round(diff, 2)))

# Sort significant differences based on absolute difference in descending order
significant_differences.sort(key=lambda x: abs(x[3]), reverse=True)

# Display list of significant differences
print("Significant Differences (Ordered by Absolute Difference):")
for feature, cond1, cond2, diff in significant_differences:
    print(f"Feature: {feature}, Conditions: {cond1} vs {cond2}, Difference: {diff}")


In [ ]:
import pandas as pd

# Read the modified CSV file
df = pd.read_csv("ModifiedFrameworkfeatures.csv")

# Define conditions with names
conditions = {
    "Experienced Rest": ("Experienced", "Rest", "experienced-rest"),
    "Experienced Meditation": ("Experienced", "Meditation", "experienced-meditation"),
    "Novice Rest": ("Novice", "Rest", "novice-rest"),
    "Novice Meditation": ("Novice", "Meditation", "novice-meditation")
}

# Initialize an empty DataFrame to store max values
condition_min_df = pd.DataFrame(columns=list(df.columns) + ['Condition'])

# Calculate max for each condition
for condition, (level, eeg_state, condition_name) in conditions.items():
    # Filter rows based on condition
    condition_df = df[(df['Level'] == level) & (df['EEG_State'] == eeg_state)]
    # Calculate max
    condition_min = condition_df.min(axis=0)
    # Set condition name
    condition_min['Condition'] = condition_name
    # Append max values to condition_max_df
    condition_min_df = condition_min_df.append(condition_min, ignore_index=True)

# Drop the columns "Filename", "EEG_state", and "Level"
condition_min_df.drop(["Filename", "EEG_State", "Level"], axis=1, inplace=True)

# Reorder columns to have "Condition" as the first column
cols = condition_min_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
condition_min_df = condition_min_df[cols]

# Save the DataFrame with condition max values to a new CSV file
condition_min_df.to_csv("ConditionMinFrameworkfeatures.csv", index=False)


In [ ]:
import numpy as np
import pandas as pd

# Load data
condition_min_df = pd.read_csv("ConditionMinFrameworkfeatures.csv", index_col=0)

# Define the conditions
conditions = {
    "Experienced Rest": ("Experienced", "Rest", "experienced-rest"),
    "Experienced Meditation": ("Experienced", "Meditation", "experienced-meditation"),
    "Novice Rest": ("Novice", "Rest", "novice-rest"),
    "Novice Meditation": ("Novice", "Meditation", "novice-meditation")
}

# Ask for input regarding the threshold percentage
threshold_percentage = float(input("Enter the threshold percentage (between 0 and 100): "))

# Convert the threshold percentage to a scale factor
threshold_scale_factor = threshold_percentage / 100

# Min-Max scaling
condition_min_df_scaled = (condition_min_df - condition_min_df.min()) / (condition_min_df.max() - condition_min_df.min())

# Perform scaling without normalization
condition_min_df_scaled *= 100  # Multiplying by 100 to keep values within a reasonable range

# Calculate the range of values for each feature
feature_ranges = condition_min_df_scaled.max() - condition_min_df_scaled.min()

# Define specified conditions to compare
specified_conditions = [
    ("Experienced Rest", "Experienced Meditation"),
    ("Experienced Rest", "Novice Rest"),
    ("Novice Rest", "Novice Meditation"),
    ("Novice Meditation", "Experienced Meditation")
]

# Find differences exceeding threshold or showing significant difference
significant_differences = []  # Store unique combinations of differences
for feature in condition_min_df.columns:
    for cond1, cond2 in specified_conditions:
        diff = condition_min_df_scaled.loc[conditions[cond1][2], feature] - condition_min_df_scaled.loc[conditions[cond2][2], feature]
        threshold = feature_ranges[feature] * threshold_scale_factor
        if abs(diff) >= threshold:
            significant_differences.append((feature, cond1, cond2, round(diff, 2)))

# Sort significant differences based on absolute difference in descending order
significant_differences.sort(key=lambda x: abs(x[3]), reverse=True)

# Display list of significant differences
print("Significant Differences (Ordered by Absolute Difference):")
for feature, cond1, cond2, diff in significant_differences:
    print(f"Feature: {feature}, Conditions: {cond1} vs {cond2}, Difference: {diff}")


In [ ]:
import pandas as pd

# Read the modified CSV file
df = pd.read_csv("ModifiedFrameworkfeatures.csv")

# Define conditions with names
conditions = {
    "Experienced Rest": ("Experienced", "Rest", "experienced-rest"),
    "Experienced Meditation": ("Experienced", "Meditation", "experienced-meditation"),
    "Novice Rest": ("Novice", "Rest", "novice-rest"),
    "Novice Meditation": ("Novice", "Meditation", "novice-meditation")
}

# Initialize an empty DataFrame to store max values
condition_median_df = pd.DataFrame(columns=list(df.columns) + ['Condition'])

# Calculate max for each condition
for condition, (level, eeg_state, condition_name) in conditions.items():
    # Filter rows based on condition
    condition_df = df[(df['Level'] == level) & (df['EEG_State'] == eeg_state)]
    # Calculate max
    condition_median = condition_df.median(axis=0)
    # Set condition name
    condition_median['Condition'] = condition_name
    # Append max values to condition_max_df
    condition_median_df = condition_median_df.append(condition_median, ignore_index=True)

# Drop the columns "Filename", "EEG_state", and "Level"
condition_median_df.drop(["Filename", "EEG_State", "Level"], axis=1, inplace=True)

# Reorder columns to have "Condition" as the first column
cols = condition_median_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
condition_median_df = condition_median_df[cols]

# Save the DataFrame with condition max values to a new CSV file
condition_median_df.to_csv("ConditionMedianFrameworkfeatures.csv", index=False)


In [ ]:
import numpy as np
import pandas as pd

# Load data
condition_median_df = pd.read_csv("ConditionMedianFrameworkfeatures.csv", index_col=0)

# Define the conditions
conditions = {
    "Experienced Rest": ("Experienced", "Rest", "experienced-rest"),
    "Experienced Meditation": ("Experienced", "Meditation", "experienced-meditation"),
    "Novice Rest": ("Novice", "Rest", "novice-rest"),
    "Novice Meditation": ("Novice", "Meditation", "novice-meditation")
}

# Ask for input regarding the threshold percentage
threshold_percentage = float(input("Enter the threshold percentage (between 0 and 100): "))

# Convert the threshold percentage to a scale factor
threshold_scale_factor = threshold_percentage / 100

# Min-Max scaling
condition_median_df_scaled = (condition_median_df - condition_median_df.min()) / (condition_median_df.max() - condition_median_df.min())

# Perform scaling without normalization
condition_median_df_scaled *= 100  # Multiplying by 100 to keep values within a reasonable range

# Calculate the range of values for each feature
feature_ranges = condition_median_df_scaled.max() - condition_median_df_scaled.min()

# Define specified conditions to compare
specified_conditions = [
    ("Experienced Rest", "Experienced Meditation"),
    ("Experienced Rest", "Novice Rest"),
    ("Novice Rest", "Novice Meditation"),
    ("Novice Meditation", "Experienced Meditation")
]

# Find differences exceeding threshold or showing significant difference
significant_differences = []  # Store unique combinations of differences
for feature in condition_median_df.columns:
    for cond1, cond2 in specified_conditions:
        diff = condition_median_df_scaled.loc[conditions[cond1][2], feature] - condition_median_df_scaled.loc[conditions[cond2][2], feature]
        threshold = feature_ranges[feature] * threshold_scale_factor
        if abs(diff) >= threshold:
            significant_differences.append((feature, cond1, cond2, round(diff, 2)))

# Sort significant differences based on absolute difference in descending order
significant_differences.sort(key=lambda x: abs(x[3]), reverse=True)

# Display list of significant differences
print("Significant Differences (Ordered by Absolute Difference):")
for feature, cond1, cond2, diff in significant_differences:
    print(f"Feature: {feature}, Conditions: {cond1} vs {cond2}, Difference: {diff}")


In [ ]:
import pandas as pd

# Read the modified CSV file
df = pd.read_csv("ModifiedFrameworkfeatures.csv")

# Define conditions with names
conditions = {
    "Experienced Rest": ("Experienced", "Rest", "experienced-rest"),
    "Experienced Meditation": ("Experienced", "Meditation", "experienced-meditation"),
    "Novice Rest": ("Novice", "Rest", "novice-rest"),
    "Novice Meditation": ("Novice", "Meditation", "novice-meditation")
}

# Initialize an empty DataFrame to store max values
condition_std_df = pd.DataFrame(columns=list(df.columns) + ['Condition'])

# Calculate max for each condition
for condition, (level, eeg_state, condition_name) in conditions.items():
    # Filter rows based on condition
    condition_std = df[(df['Level'] == level) & (df['EEG_State'] == eeg_state)]
    # Calculate max
    condition_std = condition_std.std(axis=0)
    # Set condition name
    condition_std['Condition'] = condition_name
    # Append max values to condition_max_df
    condition_std_df = condition_std_df.append(condition_std, ignore_index=True)

# Drop the columns "Filename", "EEG_state", and "Level"
condition_std_df.drop(["Filename", "EEG_State", "Level"], axis=1, inplace=True)

# Reorder columns to have "Condition" as the first column
cols = condition_std_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
condition_std_df = condition_std_df[cols]

# Save the DataFrame with condition max values to a new CSV file
condition_std_df.to_csv("ConditionStdFrameworkfeatures.csv", index=False)


In [ ]:
import numpy as np
import pandas as pd

# Load data
condition_std_df = pd.read_csv("ConditionStdFrameworkfeatures.csv", index_col=0)

# Define the conditions
conditions = {
    "Experienced Rest": ("Experienced", "Rest", "experienced-rest"),
    "Experienced Meditation": ("Experienced", "Meditation", "experienced-meditation"),
    "Novice Rest": ("Novice", "Rest", "novice-rest"),
    "Novice Meditation": ("Novice", "Meditation", "novice-meditation")
}

# Ask for input regarding the threshold percentage
threshold_percentage = float(input("Enter the threshold percentage (between 0 and 100): "))

# Convert the threshold percentage to a scale factor
threshold_scale_factor = threshold_percentage / 100

# Min-Max scaling
condition_std_df_scaled = (condition_std_df - condition_std_df.min()) / (condition_std_df.max() - condition_std_df.min())

# Perform scaling without normalization
condition_std_df_scaled *= 100  # Multiplying by 100 to keep values within a reasonable range

# Calculate the range of values for each feature
feature_ranges = condition_std_df_scaled.max() - condition_std_df_scaled.min()

# Define specified conditions to compare
specified_conditions = [
    ("Experienced Rest", "Experienced Meditation"),
    ("Experienced Rest", "Novice Rest"),
    ("Novice Rest", "Novice Meditation"),
    ("Novice Meditation", "Experienced Meditation")
]

# Find differences exceeding threshold or showing significant difference
significant_differences = []  # Store unique combinations of differences
for feature in condition_std_df.columns:
    for cond1, cond2 in specified_conditions:
        diff = condition_std_df_scaled.loc[conditions[cond1][2], feature] - condition_std_df_scaled.loc[conditions[cond2][2], feature]
        threshold = feature_ranges[feature] * threshold_scale_factor
        if abs(diff) >= threshold:
            significant_differences.append((feature, cond1, cond2, round(diff, 2)))

# Sort significant differences based on absolute difference in descending order
significant_differences.sort(key=lambda x: abs(x[3]), reverse=True)

# Display list of significant differences
print("Significant Differences (Ordered by Absolute Difference):")
for feature, cond1, cond2, diff in significant_differences:
    print(f"Feature: {feature}, Conditions: {cond1} vs {cond2}, Difference: {diff}")
